In [ ]:
def registrar_usuario(nombre, apellido, documento, placa):
    dic_usuario = {}
    errores = []

    # Validar nombre
    if len(nombre) < 3:
        errores.append("El nombre no puede tener menos de 3 letras")
    if any(letra.isdigit() for letra in nombre):
        errores.append("El nombre no puede tener números")

    # Validar apellido
    if len(apellido) < 3:
        errores.append("El apellido no puede tener menos de 3 letras")
    if any(letra.isdigit() for letra in apellido):
        errores.append("El apellido no puede tener números")

    # Validar documento
    if len(documento) < 3 or len(documento) > 15:
        errores.append("El documento debe tener entre 3 y 15 dígitos")
    if not documento.isdigit():
        errores.append("El documento solo debe contener números")

    # Validar placa
    if len(placa) != 6:
        errores.append("La placa debe tener exactamente 6 caracteres")
    else:
        if not placa[:3].isalpha():
            errores.append("La placa debe tener letras en las primeras 3 posiciones")
        if not placa[3:].isdigit():
            errores.append("La placa debe tener números en las últimas 3 posiciones")

    if errores:
        return False, errores

    dic_usuario["nombre"] = nombre
    dic_usuario["apellido"] = apellido
    dic_usuario["documento"] = documento
    dic_usuario["placa"] = placa.upper()

    return True, dic_usuario

#INGRESO VEHICULO

# Lista global de usuarios
lista_usuarios_registrados = []

# Crear los 50 espacios disponibles
espacios_disponibles = [f"P{n+1}" for n in range(50)]

# Diccionario que representa el estado actual del parqueadero
lista_parqueadero = []
dic_parqueadero={}

import datetime
import random
import csv

def ingresar_vehiculo(lista_usuarios_registrados, parqueadero, espacios_disponibles):
    print("\n-------- Ingreso de vehículo --------")
    documento = input("Ingrese su documento: ")
    placa = input("Ingrese la placa del vehículo: ").upper()

    # Buscar el usuario
    usuario = next((user for user in lista_usuarios_registrados if user["documento"] == documento and user["placa"] == placa), None)

    if not usuario:
        print("Usuario o vehículo no registrado.")
        return

    if placa in parqueadero:
        print("Este vehículo ya se encuentra dentro del parqueadero.")
        return

    if not espacios_disponibles:
        print("No hay espacios disponibles en el parqueadero.")
        return

    espacio_asignado = espacios_disponibles.pop(random.randint(0,51)) 
    hora_ingreso = datetime.datetime.now()

    dic_parqueadero = {
    'Placa': placa,
    'Usuario': usuario,
    'Espacio asignado': espacio_asignado,
    'Hora de ingreso': hora_ingreso.strftime('%Y-%m-%d %H:%M:%S')
    }

    lista_parqueadero.append(dic_parqueadero)

    # Recibo
    print("\n--------- Recibo de ingreso ---------")
    print(f"Nombre: {usuario['nombre'].capitalize()} {usuario['apellido'].capitalize()}")
    print(f"Documento: {usuario['documento']}")
    print(f"Placa: {placa}")
    print(f"Espacio asignado: {espacio_asignado}")
    print(f"Hora de ingreso: {hora_ingreso.strftime('%Y-%m-%d %H:%M:%S')}")
    print("-------------------------------------")


    print('\nRegistro de asignación de celdas')
    return lista_parqueadero
    ingresar_vehiculo(lista_nuevos_registros, lista_parqueadero, espacios_disponibles)

#RETIRO VEHICULO
lista_vehiculos_retirados=[]

def retirar_vehiculo(lista_parqueadero, espacios_disponibles, lista_vehiculos_retirados):
    print("\n--- Retiro de vehículo ---")
    placa = input("Ingrese la placa del vehículo: ").upper()
    registro_retiro={}

    # Buscar el registro del vehículo
    registro = next((vehiculo for vehiculo in lista_parqueadero if vehiculo['Placa'] == placa), None)

    if not registro:
        print("El vehículo no se encuentra en el parqueadero.")
        return

    hora_ingreso_str = registro["Hora de ingreso"]
    hora_ingreso = datetime.datetime.strptime(hora_ingreso_str, "%Y-%m-%d %H:%M:%S")
    hora_salida = datetime.datetime.now()
    tiempo_total = hora_salida - hora_ingreso

    # Calcular tiempo total
    minutos_totales = int(tiempo_total.total_seconds() // 60)
    horas = minutos_totales // 60
    minutos_restantes = minutos_totales % 60
    cuartos = minutos_restantes // 15

    # Calcular cobros
    cobro_horas = horas * 7000
    cobro_cuartos = cuartos * 1500
    total = cobro_horas + cobro_cuartos
    if total < 7000:
        total = 7000

    # Liberar espacio
    espacio = registro["Espacio asignado"]
    espacios_disponibles.append(espacio)
    lista_parqueadero.remove(registro)

    # Crear el diccionario del retiro
    registro_retiro = {
        "placa": placa,
        "espacio_liberado": espacio,
        "hora_ingreso": hora_ingreso.strftime('%Y-%m-%d %H:%M:%S'),
        "hora_salida": hora_salida.strftime('%Y-%m-%d %H:%M:%S'),
        "tiempo_total": f"{horas}h {minutos_restantes}min",
        "cobro_horas": cobro_horas,
        "cobro_cuartos": cobro_cuartos,
        "total_pagado": total
    }

    # Agregar a la lista acumulada
    lista_vehiculos_retirados.append(registro_retiro)

    # Mostrar factura
    print("\n--------- Factura de Salida ---------")
    print(f"Placa: {placa}")
    print(f"Espacio liberado: {espacio}")
    print(f"Hora de ingreso: {hora_ingreso.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Hora de salida: {hora_salida.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Tiempo total: {horas} h y {minutos_restantes} min")
    print(f"Cobro por horas: ${cobro_horas}")
    print(f"Cobro por cuartos de hora: ${cobro_cuartos}")
    print(f"Total a pagar: ${total}")
    print("-------------------------------------")

    print('\n Registro de vehículos retirados guardado.')

    # ADMINISTRADOR

usuarios = []
vehiculos_en_parqueo = []
vehiculos_retirados = []
celdas = [None] * 50
admin_credenciales = {"UPARKING123": "123456"}  # Aquí definimos el usuario y contraseña correctos

def administrador(usuario, contrasena):

    if admin_credenciales.get(usuario) != contrasena:
        return False, "Usuario o contraseña incorrectos."

    total_registrados = len(lista_usuarios_registrados)
    total_retirados = len(lista_vehiculos_retirados)
    total_sin_retirar = len(lista_parqueadero)

    total_pago = sum(vehiculo['total_pagado'] for vehiculo in lista_vehiculos_retirados)

    if total_retirados > 0:
        tiempos = []
        for v in lista_vehiculos_retirados:
            h_ingreso = datetime.datetime.strptime(v['hora_ingreso'], "%Y-%m-%d %H:%M:%S")
            h_salida = datetime.datetime.strptime(v['hora_salida'], "%Y-%m-%d %H:%M:%S")
            tiempos.append((h_salida - h_ingreso).total_seconds() / 60)

        tiempo_promedio = sum(tiempos) / len(tiempos)
    else:
        tiempo_promedio = 0

    vehiculo_max = max(lista_vehiculos_retirados, key=lambda v: datetime.datetime.strptime(v['hora_salida'], "%Y-%m-%d %H:%M:%S") - datetime.datetime.strptime(v['hora_ingreso'], "%Y-%m-%d %H:%M:%S"), default=None)
    vehiculo_min = min(lista_vehiculos_retirados, key=lambda v: datetime.datetime.strptime(v['hora_salida'], "%Y-%m-%d %H:%M:%S") - datetime.datetime.strptime(v['hora_ingreso'], "%Y-%m-%d %H:%M:%S"), default=None)

    ocupacion = len(lista_parqueadero)

    reporte = f"""
--- Panel de Administración ---
 Total vehículos registrados: {total_registrados}
 Total vehículos retirados: {total_retirados}
 Vehículos sin retirar: {total_sin_retirar}
 Total pagos recibidos: ${total_pago}
 Tiempo promedio de estancia: {tiempo_promedio:.2f} minutos
 Lista de usuarios registrados:
"""
    for u in lista_usuarios_registrados:
        reporte += f"- {u['nombre']} {u['apellido']} | Documento: {u['documento']} | Placa: {u['placa']}\n"

    if vehiculo_max and vehiculo_min:
        reporte += f"\n Vehículo con mayor tiempo de parqueo: {vehiculo_max['placa']} ({vehiculo_max['tiempo_total']})"
        reporte += f"\n Vehículo con menor tiempo de parqueo: {vehiculo_min['placa']} ({vehiculo_min['tiempo_total']})"

    reporte += f"\n Celdas ocupadas actualmente: {ocupacion}"

    return True, reporte

    
# Menú
def menu():
    while True:
        print("-------------------------------------------------")
        print("                    UPARKING                     ")
        print("-------------------------------------------------")
        print("\n MENÚ PRINCIPAL")
        print("1. Registrar usuario")
        print("2. Ingresar vehículo")
        print("3. Retirar vehículo")
        print("4. Administrador")
        print("5. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            nombre = input("Ingrese el nombre: ")
            apellido = input("Ingrese el apellido: ")
            documento = input("Ingrese el documento: ")
            placa = input("Ingrese la placa (ABC123): ")

            exito, resultado = registrar_usuario(nombre, apellido, documento, placa)

            if exito:
                lista_usuarios_registrados.append(resultado)
                print(" Usuario registrado exitosamente.")
            else:
                print(" Errores encontrados:")
                for error in resultado:
                    print(" -", error)

        elif opcion == "2":
            print("\n INGRESO VEHÍCULO")
            ingresar_vehiculo(lista_usuarios_registrados, lista_parqueadero, espacios_disponibles)

            

        elif opcion == "3":
            print("\n RETIRO VEHÍCULO")
            retirar_vehiculo(lista_parqueadero, espacios_disponibles, lista_vehiculos_retirados)

        elif opcion == "4":
            print("\n ACCESO ADMINISTRADOR")
            usuario = input("Usuario administrador: ")
            contraseña = input("Contraseña: ")
            acceso, reporte = administrador(usuario, contraseña)
            if acceso:
                print(reporte)
            else:
                print("Acceso denegado: " + reporte)

        elif opcion == "5":
            exportar = input("¿Desea exportar un resumen del parqueadero a un archivo CSV? (si/no: ").lower()
            if exportar == "si":
                with open("resumen_parqueadero.csv", mode="w", newline="", encoding="utf-8") as archivo_csv:
                    writer = csv.writer(archivo_csv)

                    writer.writerow(["--- RESUMEN DEL PARQUEADERO ---"])
                    writer.writerow(["Total usuarios registrados", len(lista_usuarios_registrados)])
                    writer.writerow(["Total vehiculos en parqueadero", len(lista_parqueadero)])
                    writer.writerow(["Total vehiculos retirados", len(lista_vehiculos_retirados)])

                    total_pago = sum(v['total_pagado'] for v in lista_vehiculos_retirados)
                    writer.writerow(["Total dinero recibido", f"${total_pago}"])

                    # Calcular tiempo promedio
                    tiempos = []
                    for v in lista_vehiculos_retirados:
                        h_ing = datetime.datetime.strptime(v['hora_ingreso'], "%Y-%m-%d %H:%M:%S")
                        h_sal = datetime.datetime.strptime(v['hora_salida'], "%Y-%m-%d %H:%M:%S")
                        minutos = (h_sal - h_ing).total_seconds() / 60
                        tiempos.append(minutos)

                    if tiempos:
                        promedio = round(sum(tiempos) / len(tiempos), 2)
                    else:
                        promedio = 0

                    writer.writerow(["Tiempo promedio de parqueo (minutos)", promedio])
                    writer.writerow([])

                    # Detalle de usuarios
                    writer.writerow(["Usuarios registrados:"])
                    writer.writerow(["Nombre", "Apellido", "Documento", "Placa"])
                    for u in lista_usuarios_registrados:
                        writer.writerow([u['nombre'], u['apellido'], u['documento'], u['placa']])
                    writer.writerow([])

                    # Detalle de vehículos retirados
                    writer.writerow(["Vehículos retirados:"])
                    writer.writerow(["Placa", "Espacio", "Ingreso", "Salida", "Tiempo", "Horas $", "Cuartos $", "Total $"])
                    for v in lista_vehiculos_retirados:
                        writer.writerow([
                            v['placa'], v['espacio_liberado'], v['hora_ingreso'], v['hora_salida'],
                            v['tiempo_total'], v['cobro_horas'], v['cobro_cuartos'], v['total_pagado']
                        ])

                print("✅ Archivo 'resumen_parqueadero.csv' exportado exitosamente.")
            else:
                print("No se exportaron los datos.")
            print(" Saliendo del sistema. ¡Gracias por usar UPARKING!")
            break


        else:
            print(" Opción no válida. Intente nuevamente.") 


# Ejecutar menú
if __name__ == "__main__":
    menu()

-------------------------------------------------
                    UPARKING                     
-------------------------------------------------

 MENÚ PRINCIPAL
1. Registrar usuario
2. Ingresar vehículo
3. Retirar vehículo
4. Administrador
5. Salir
 Usuario registrado exitosamente.
-------------------------------------------------
                    UPARKING                     
-------------------------------------------------

 MENÚ PRINCIPAL
1. Registrar usuario
2. Ingresar vehículo
3. Retirar vehículo
4. Administrador
5. Salir

 INGRESO VEHÍCULO

-------- Ingreso de vehículo --------

--------- Recibo de ingreso ---------
Nombre: Jose Guerra
Documento: 12345
Placa: ABC123
Espacio asignado: P13
Hora de ingreso: 2025-06-26 12:23:53
-------------------------------------

Registro de asignación de celdas
-------------------------------------------------
                    UPARKING                     
-------------------------------------------------

 MENÚ PRINCIPAL
1. Registr